In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.db.models import Sum, Max, Subquery, Q, F, Avg, Count, Min, Max, Sum, F, Q
from django.db.models import Value, IntegerField, When, FloatField
from django.db.models.functions import Coalesce, Greatest, Floor
from account.models import User
from booth.models import Booth, Transaction, Participation
from player.models import Player
import random
import pandas as pd


In [2]:
User.objects.values('player').annotate(
    health_score=Max('player__born_health_score') + Coalesce(Sum('player__participation_player__health_score'), Value(0.0)),
    skill_score=Max('player__born_skill_score') + Sum('player__participation_player__skill_score'),
    growth_score=Max('player__born_growth_score') + Sum('player__participation_player__growth_score'),
    relationship_score=Max('player__born_relationship_score') + Sum('player__participation_player__relationship_score'),
)

FieldError: Expression contains mixed types: IntegerField, FloatField. You must set output_field.

In [32]:
pay = Case(
    When(transaction_player__type='pay', then='transaction_player__money'), 
    default=Value(0.0),
    output_field = FloatField()
)
receive = Case(
    When(transaction_player__type='receive', then='transaction_player__money'), 
    default=Value(0.0),
    output_field = FloatField()
)
deposit = Case(
    When(transaction_player__type='deposit', then='transaction_player__money'), 
    default=Value(0.0),
    output_field = FloatField()
)
withdrawal = Case(
    When(transaction_player__type='withdrawal', then='transaction_player__money'), 
    default=Value(0.0),
    output_field = FloatField()
)
withdrawal_with_interest = Case(
    When(transaction_player__type='withdrawal', then=F('transaction_player__money') * (Value(1.0) + F('transaction_player__interest_rate'))), 
    default=Value(0.0),
    output_field = FloatField()
)
Player.objects.filter(Q(user__user_type='student'), Q(active=True)).values('user').annotate(
    total_wealth = Sum(F('born_money') + receive + withdrawal_with_interest - pay - deposit)
).order_by('-total_wealth')[:10]

<QuerySet [{'user': 101, 'total_wealth': 113800.0}, {'user': 397, 'total_wealth': 50000.0}, {'user': 169, 'total_wealth': 50000.0}, {'user': 440, 'total_wealth': 50000.0}, {'user': 433, 'total_wealth': 50000.0}, {'user': 456, 'total_wealth': 50000.0}, {'user': 196, 'total_wealth': 50000.0}, {'user': 482, 'total_wealth': 50000.0}, {'user': 437, 'total_wealth': 50000.0}, {'user': 331, 'total_wealth': 50000.0}]>

In [8]:
health_score = Case(
    When(participation_player__health_score='pay', then='transaction_player__money'), 
    default=Value(0.0),
    output_field = FloatField()
)
receive = Case(
    When(transaction_player__type='receive', then='transaction_player__money'), 
    default=Value(0.0),
    output_field = FloatField()
)
deposit = Case(
    When(transaction_player__type='deposit', then='transaction_player__money'), 
    default=Value(0.0),
    output_field = FloatField()
)
withdrawal = Case(
    When(transaction_player__type='withdrawal', then='transaction_player__money'), 
    default=Value(0.0),
    output_field = FloatField()
)
withdrawal_with_interest = Case(
    When(transaction_player__type='withdrawal', then=F('transaction_player__money') * (Value(1.0) + F('transaction_player__interest_rate'))), 
    default=Value(0.0),
    output_field = FloatField()
)
Player.objects.filter(Q(user__user_type='student'), Q(active=True)).values('user').annotate(
    mark = Max(F('born_health_score')) \
            + Sum(Coalesce('participation_player__health_score', Value(0))) \
            + Max(F('born_skill_score')) \
            + Sum(Coalesce('participation_player__skill_score', Value(0))) \
            + Max(F('born_growth_score')) + Sum(Coalesce('participation_player__growth_score', Value(0))) \
            + Max(F('born_relationship_score')) \
            + Sum(Coalesce('participation_player__relationship_score', Value(0)))
).order_by('-mark')[:10]

<QuerySet [{'user': 101, 'mark': 955}, {'user': 161, 'mark': 365}, {'user': 298, 'mark': 355}, {'user': 392, 'mark': 355}, {'user': 159, 'mark': 345}, {'user': 451, 'mark': 335}, {'user': 263, 'mark': 330}, {'user': 359, 'mark': 325}, {'user': 424, 'mark': 325}, {'user': 345, 'mark': 325}]>